Importing Libraries

In [ ]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install neattext
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install Keras-Preprocessing

Importing dataset

In [ ]:
data=pd.read_csv('Suicide_Mentalhealth_Detection.csv')
data.head()

In [ ]:
data['class'].value_counts()

In [ ]:
data['class'].value_counts().index.values

Splitting Dataset Into Train And Test

In [ ]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [ ]:
train_data['class'].value_counts().index.values

In [ ]:
plx.bar(train_data,x=train_data['class'].value_counts().index.values,
        y=train_data['class'].value_counts(),color=['Suicide','Not Suicide'])

Data Cleaning

In [ ]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [ ]:
train_data

In [ ]:
train_data.text

In [ ]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [ ]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

Preprocessing

In [ ]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [ ]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['class'])
test_output=lbl_target.transform(test_data['class'])

In [ ]:
train_output

Embeddings

In [ ]:

with open('/content/drive/MyDrive/Colab Notebooks/glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [ ]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [ ]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

Model Training And Testing

In [ ]:
model=Sequential()
model.add(Input(shape=(50,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

In [ ]:
test_output

In [ ]:
y_pred = model.predict(test_text_pad)

In [ ]:
y_pred = (y_pred > 0.5)
y_pred = 1*y_pred

y_pred

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_output, y_pred).ravel()
false_positive_rate = fp / (fp + tn)

print("False Positive Rate:", false_positive_rate)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_output, y_pred)

print(accuracy)

In [ ]:
len(y_pred)


In [ ]:
len(test_output)

In [ ]:
model.save('model.h5')

In [ ]:
tokenizer.save('tokenize.h5')

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('token.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
text = ["i am feeling lonely"]

In [ ]:
cleaned_text,text_length=clean_text(text)
text_seq=tokenizer.texts_to_sequences(cleaned_text)
text_pad=pad_sequences(text_seq,maxlen=50)

In [ ]:
model.predict(text_pad)

In [ ]:
lbl_target.fit_transform(model.predict(text_pad))

In [ ]:
model.accuracy()